Construct vocabulary tables, based on tables downloaded from Athena website and available here on s3://hls-eng-data-public/data/rwe/omop-vocabs/ If you like to download a different dataset, downoad the vocabularies from Athena and use databricks dbfs api utilities to upload downloaded vocabularies to dbfs under your vocab_path.

In [ ]:
import os
import subprocess
import sys
from typing import List
from pyspark.sql import SparkSession
import logging


# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the base directory


def create_spark_session(app_name="EHR Data Loader", aws_access_key=None, aws_secret_key=None):
    """
    Create and return a Spark session configured for Delta Lake with S3 access.
    """
    # Stop any existing Spark session
    try:
        SparkSession.builder.getOrCreate().stop()
        print("Stopped existing Spark session")
    except:
        print("No existing Spark session to stop")


    # Get the absolute path two levels up from the current file's directory
    jars_home = '/home/developer/projects/delta-spark-handbook/delta-jars'

    print(f"Jars home is set to: {jars_home}")

    # Define the base directory for JAR files
    if not os.path.exists(jars_home):
        raise Exception(f"JAR directory not found at: {jars_home}")

    # Required JARs
    jar_locations = [
        f"{jars_home}/delta-spark_2.12-3.3.0.jar",
        f"{jars_home}/delta-storage-3.3.0.jar",
        f"{jars_home}/hadoop-aws-3.3.4.jar",
        f"{jars_home}/bundle-2.24.12.jar",
        # Add Hadoop client JARs
        f"{jars_home}/hadoop-client-3.4.1.jar",
        f"{jars_home}/hadoop-client-runtime-3.4.1.jar",
        f"{jars_home}/hadoop-client-api-3.4.1.jar"
    ]

    # Verify all JARs exist
    for jar in jar_locations:
        if not os.path.exists(jar):
            raise Exception(f"Required JAR not found: {jar}")

    # Create Hadoop configuration directory
    hadoop_conf_dir = "hadoop-conf"
    os.makedirs(hadoop_conf_dir, exist_ok=True)

    # Write core-site.xml with S3 configuration
    core_site_xml = f"""<?xml version="1.0"?>
<configuration>
    <property>
        <name>fs.s3a.impl</name>
        <value>org.apache.hadoop.fs.s3a.S3AFileSystem</value>
    </property>
    <property>
        <name>fs.s3a.endpoint</name>
        <value>http://localhost:9000</value>
    </property>
    <property>
        <name>fs.s3a.access.key</name>
        <value>{aws_access_key or 'minioadmin'}</value>
    </property>
    <property>
        <name>fs.s3a.secret.key</name>
        <value>{aws_secret_key or 'minioadmin'}</value>
    </property>
    <property>
        <name>fs.s3a.path.style.access</name>
        <value>true</value>
    </property>
    <property>
        <name>fs.s3a.connection.ssl.enabled</name>
        <value>false</value>
    </property>
</configuration>"""

    with open(f"{hadoop_conf_dir}/core-site.xml", "w") as f:
        f.write(core_site_xml)

    # Set environment variables
    os.environ["HADOOP_CONF_DIR"] = os.path.abspath(hadoop_conf_dir)
    os.environ["SPARK_HOME"] = "/opt/spark"
    os.environ["SPARK_CLASSPATH"] = ":".join(
        [os.path.abspath(jar) for jar in jar_locations])
    os.environ["HADOOP_CLASSPATH"] = os.environ["SPARK_CLASSPATH"]

    # Create Spark session with comprehensive configuration
    builder = (SparkSession.builder
               .appName(app_name)
               .master("local[*]")
               .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
               .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
               .config("spark.sql.catalogImplementation", "hive")
               .config("spark.hadoop.javax.jdo.option.ConnectionURL", "jdbc:postgresql://localhost:5432/metastore_db")
               .config("spark.hadoop.javax.jdo.option.ConnectionDriverName", "org.postgresql.Driver")
               .config("spark.hadoop.javax.jdo.option.ConnectionUserName", "admin")
               .config("spark.hadoop.javax.jdo.option.ConnectionPassword", "admin")
               .config("spark.hadoop.hive.metastore.uris", "thrift://localhost:9083")
               .config("spark.sql.warehouse.dir", "s3a://wba/warehouse")
               .config("spark.driver.extraClassPath", ":".join([os.path.abspath(jar) for jar in jar_locations]))
               .config("spark.executor.extraClassPath", ":".join([os.path.abspath(jar) for jar in jar_locations]))
               .config("spark.jars.excludes", "org.slf4j:slf4j-log4j12,org.slf4j:slf4j-reload4j,org.slf4j:log4j-slf4j-impl")
               .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
               .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
               .config("spark.hadoop.fs.s3a.access.key", aws_access_key or "minioadmin")
               .config("spark.hadoop.fs.s3a.secret.key", aws_secret_key or "minioadmin")
               .config("spark.hadoop.fs.s3a.path.style.access", "true")
               .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
               .config("spark.hadoop.fs.s3a.fast.upload", "true")
               .config("spark.hadoop.fs.s3a.multipart.size", "5242880")
               .config("spark.hadoop.fs.s3a.block.size", "5242880")
               .config("spark.hadoop.fs.s3a.multipart.threshold", "5242880")
               .config("spark.hadoop.fs.s3a.threads.core", "10")
               .config("spark.hadoop.fs.s3a.threads.max", "20")
               .config("spark.hadoop.fs.s3a.max.total.tasks", "50")
               .config("spark.hadoop.fs.s3a.connection.timeout", "60000")
               .config("spark.hadoop.fs.s3a.connection.establish.timeout", "60000")
               .config("spark.hadoop.fs.s3a.socket.timeout", "60000")
               .config("spark.hadoop.fs.s3a.connection.maximum", "50")
               .config("spark.hadoop.fs.s3a.fast.upload.buffer", "bytebuffer")
               .config("spark.hadoop.fs.s3a.fast.upload.active.blocks", "2")
               .config("spark.hadoop.fs.s3a.multipart.purge", "false")
               .config("spark.hadoop.fs.s3a.multipart.purge.age", "86400000")
               .config("spark.hadoop.fs.s3a.retry.limit", "10")
               .config("spark.hadoop.fs.s3a.retry.interval", "1000")
               .config("spark.hadoop.fs.s3a.attempts.maximum", "10")
               .config("spark.hadoop.fs.s3a.connection.request.timeout", "60000")
               .config("spark.hadoop.fs.s3a.threads.keepalivetime", "60000")
               .enableHiveSupport())

    return builder.getOrCreate()

aws_access_key = "minioadmin"
aws_secret_key = "minioadmin"

spark = create_spark_session(
    app_name="OMOP Vocab Setup", aws_access_key=aws_access_key, aws_secret_key=aws_secret_key)

spark.sql("SHOW DATABASES").show()

In [ ]:
import boto3
from urllib.parse import urlparse
from botocore import UNSIGNED
from botocore.config import Config

# Variables
database_name = "omop531"
project_name = 'omop-cdm-100K'
vocab_s3_path = "s3://hls-eng-data-public/data/rwe/omop-vocabs/"

# Output
print(f"Using OMOP version {database_name}")
print(f"Using vocabulary tables in {vocab_s3_path}")

# Set the database in Spark
spark.sql(f"USE {database_name}")

# Parse S3 path
parsed = urlparse(vocab_s3_path)
bucket = parsed.netloc
prefix = parsed.path.lstrip('/')

# Initialize S3 client
s3 = boto3.client('s3', config=boto3.session.Config(signature_version=UNSIGNED))

# List objects in S3 prefix
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

# Display the list of files
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No files found in the specified S3 path.")


In [8]:
from pyspark.sql.functions import to_date
import tempfile, os, io, gzip, time
from urllib.parse import urlparse
import boto3
from botocore import UNSIGNED
from botocore.config import Config

# -----------------------------------
# Spark legacy behavior for OMOP dates
# -----------------------------------
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
spark.sql("set spark.sql.legacy.parquet.datetimeRebaseModeInWrite=LEGACY")

# -----------------------------------
# Config
# -----------------------------------
database_name = "omop531"
vocab_s3_path = "s3://hls-eng-data-public/data/rwe/omop-vocabs/"
output_path_base = "s3a://my-minio-bucket/omop-vocab-delta"

parsed = urlparse(vocab_s3_path)
bucket = parsed.netloc
prefix = parsed.path.lstrip('/').rstrip('/')

# -----------------------------------
# Get public file list via boto3
# -----------------------------------
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
available_keys = [obj['Key'] for obj in response.get('Contents', [])]

# -----------------------------------
# OMOP tables
# -----------------------------------
tablelist = [
    "DOMAIN", "CONCEPT", "VOCABULARY", "CONCEPT_ANCESTOR", "CONCEPT_RELATIONSHIP",
    "RELATIONSHIP", "CONCEPT_SYNONYM", "CONCEPT_CLASS", "DRUG_STRENGTH"
]
 
# -----------------------------------
# Main loop using Spark
# -----------------------------------
for table_name in tablelist:
    print(f"\n📦 Processing {table_name}...")

    key = f"{prefix}/{table_name}.csv.gz"
    if key not in available_keys:
        print(f"❌ File not found in S3: {key}")
        continue

    start_time = time.time()
    try:
        print("⏳ Downloading from S3...")
        response = s3.get_object(Bucket=bucket, Key=key)
        gz_content = response['Body'].read()
        download_time = time.time()
        print(f"✅ Download complete in {download_time - start_time:.2f}s")

        # Save raw .csv.gz file to MinIO
        raw_gz_key = key
        print(f"📤 Saving raw .csv.gz to MinIO: {raw_gz_key}...")
        s3_minio = boto3.client(
            "s3",
            endpoint_url="http://localhost:9000",  # adjust to your MinIO endpoint
            aws_access_key_id="minioadmin",
            aws_secret_access_key="minioadmin"
        )
        s3_minio.put_object(
            Bucket=f"{database_name}",
            Key=raw_gz_key,
            Body=gz_content
        )
        print("✅ Raw .csv.gz saved to MinIO.")
        
        # Use decompression + in-memory IO stream to avoid temp file I/O
        with gzip.GzipFile(fileobj=io.BytesIO(gz_content)) as gz:
            content = gz.read().decode('utf-8')

        with tempfile.NamedTemporaryFile(suffix=".csv", delete=False, mode='w', encoding='utf-8') as tmp_file:
            tmp_file.write(content)
            tmp_path = tmp_file.name
        
        print("⏳ Reading CSV with Spark...")
        
        df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "false") \
            .option("mode", "PERMISSIVE") \
            .option("quote", "\"") \
            .option("escape", "\"") \
            .csv(tmp_path)  
     
        read_time = time.time()
        print(f"✅ Spark read complete in {read_time - download_time:.2f}s")

        output_path_base = f"s3a://{database_name}"


        # Handle date columns
        if table_name in ["CONCEPT", "CONCEPT_RELATIONSHIP", "DRUG_STRENGTH"]:
            df = df.withColumn("valid_start_date", to_date("valid_start_date", "yyyy-MM-dd")) \
                   .withColumn("valid_end_date", to_date("valid_end_date", "yyyy-MM-dd"))

        # Get warehouse dir from Spark config
        warehouse_dir = spark.conf.get("spark.sql.warehouse.dir").rstrip("/")

        table_name = table_name.lower()
        # Format: {warehouse_dir}/{database}.db/{table_name}
        table_path = f"{warehouse_dir}/{database_name}.db/{table_name}"

        print(f"⏳ Writing to Delta format: {table_path}...")
        df.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(f"{table_path}")
        write_time = time.time()
        print(f"✅ Delta write complete in {write_time - read_time:.2f}s")

        print(f"⏳ Registering Delta table:  {database_name}.{table_name}...")
 
        spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {database_name}.{table_name}
        USING DELTA
            LOCATION '{table_path}'
        """)

        register_time = time.time()
        print(f"✅ Successfully registered {table_path} into table  {database_name}.{table_name}")

        os.remove(tmp_path)
        
        total_time = register_time - start_time
        print(f"✅ Completed {table_name} in {total_time:.2f}s → {database_name}.{table_name}")

    except Exception as e:
        print(f"❌ Error processing {table_name}: {e}")


25/03/30 17:53:42 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInWrite' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInWrite' instead.



📦 Processing DOMAIN...
⏳ Downloading from S3...
✅ Download complete in 0.10s
📤 Saving raw .csv.gz to MinIO: data/rwe/omop-vocabs/DOMAIN.csv.gz...
✅ Raw .csv.gz saved to MinIO.
⏳ Reading CSV with Spark...
✅ Spark read complete in 0.17s
⏳ Writing to Delta format: s3a://wba/warehouse/omop531.db/domain...


✅ Delta write complete in 2.95s
⏳ Registering Delta table:  omop531.domain...
✅ Successfully registered s3a://wba/warehouse/omop531.db/domain into table  omop531.domain
✅ Completed domain in 3.27s → omop531.domain

📦 Processing CONCEPT...
⏳ Downloading from S3...
✅ Download complete in 5.32s
📤 Saving raw .csv.gz to MinIO: data/rwe/omop-vocabs/CONCEPT.csv.gz...
✅ Raw .csv.gz saved to MinIO.
⏳ Reading CSV with Spark...
✅ Spark read complete in 3.74s
⏳ Writing to Delta format: s3a://wba/warehouse/omop531.db/concept...


25/03/30 17:53:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


✅ Delta write complete in 29.04s
⏳ Registering Delta table:  omop531.concept...
✅ Successfully registered s3a://wba/warehouse/omop531.db/concept into table  omop531.concept
✅ Completed concept in 38.15s → omop531.concept

📦 Processing VOCABULARY...
⏳ Downloading from S3...
✅ Download complete in 0.53s
📤 Saving raw .csv.gz to MinIO: data/rwe/omop-vocabs/VOCABULARY.csv.gz...
✅ Raw .csv.gz saved to MinIO.
⏳ Reading CSV with Spark...
✅ Spark read complete in 0.24s
⏳ Writing to Delta format: s3a://wba/warehouse/omop531.db/vocabulary...


...

25/03/30 18:00:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


✅ Delta write complete in 29.79s
⏳ Registering Delta table:  omop531.concept_synonym...
✅ Successfully registered s3a://wba/warehouse/omop531.db/concept_synonym into table  omop531.concept_synonym
✅ Completed concept_synonym in 47.11s → omop531.concept_synonym

📦 Processing CONCEPT_CLASS...
⏳ Downloading from S3...
✅ Download complete in 0.59s
📤 Saving raw .csv.gz to MinIO: data/rwe/omop-vocabs/CONCEPT_CLASS.csv.gz...
✅ Raw .csv.gz saved to MinIO.
⏳ Reading CSV with Spark...
✅ Spark read complete in 0.21s
⏳ Writing to Delta format: s3a://wba/warehouse/omop531.db/concept_class...


✅ Delta write complete in 4.02s
⏳ Registering Delta table:  omop531.concept_class...
✅ Successfully registered s3a://wba/warehouse/omop531.db/concept_class into table  omop531.concept_class
✅ Completed concept_class in 4.96s → omop531.concept_class

📦 Processing DRUG_STRENGTH...
⏳ Downloading from S3...
✅ Download complete in 1.75s
📤 Saving raw .csv.gz to MinIO: data/rwe/omop-vocabs/DRUG_STRENGTH.csv.gz...
✅ Raw .csv.gz saved to MinIO.
⏳ Reading CSV with Spark...
✅ Spark read complete in 1.38s
⏳ Writing to Delta format: s3a://wba/warehouse/omop531.db/drug_strength...


25/03/30 18:01:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


✅ Delta write complete in 33.03s
⏳ Registering Delta table:  omop531.drug_strength...
✅ Successfully registered s3a://wba/warehouse/omop531.db/drug_strength into table  omop531.drug_strength
✅ Completed drug_strength in 36.56s → omop531.drug_strength
